In [ ]:
# ActionChains Documentation: 
# https://www.selenium.dev/selenium/docs/api/py/webdriver/selenium.webdriver.common.action_chains.html#selenium.webdriver.common.action_chains.ActionChains.release

# Beautiful Soup Docs: https://www.crummy.com/software/BeautifulSoup/bs4/doc/


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd

# Set up Selenium
options = webdriver.ChromeOptions()
options.headless = True  # Run Chrome in headless mode (without GUI)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Visit Vivino's page
url = "https://www.vivino.com/explore"
driver.get(url)

# Wait for the page to load properly
wait = WebDriverWait(driver, 10)

# Step 1: click the dropdown menu
dropdown = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "simpleLabel-module__selectedKey--3ngzL")))
dropdown.click()
time.sleep(2)

# Step 2: select "United States" from the list
country_option = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@data-value='us']")))
country_option.click()
time.sleep(3)

# Step 3: unselect the "Red" wine filter
red_wine_label = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(@class, 'MuiButtonBase-root') and @data-testid='wineTypes_1']")))
driver.execute_script("arguments[0].click();", red_wine_label)
time.sleep(3)

# Step 4: drag slider to select all prices.
# Find the first slider handle (left)
slider_1 = driver.find_element(By.XPATH, "//div[@class='rc-slider-handle rc-slider-handle-1']")

# Use ActionChains to drag the first slider
actions = ActionChains(driver)
actions.click_and_hold(slider_1).move_by_offset(-10, 0).release().perform()

# Find the second slider handle (right)
slider_2 = driver.find_element(By.XPATH, "//div[@class='rc-slider-handle rc-slider-handle-2']")

# Use ActionChains to drag the second slider
actions = ActionChains(driver)
actions.click_and_hold(slider_2).move_by_offset(150, 0).release().perform()

# Step 5: Select Any Rating option
any_rating_label = wait.until(EC.element_to_be_clickable((By.XPATH, "//label[.//input[@name='rating' and @value='1']]")))
driver.execute_script("arguments[0].click();", any_rating_label)


# Step 6: Select Countries
countries_dropdown = driver.find_element(By.XPATH, "//legend[@data-testid='filter-toggle-button'][.//h5[.='Countries']]")
driver.execute_script("arguments[0].click();", countries_dropdown)
time.sleep(2)
countries_ar = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(@class, 'MuiButtonBase-root')]/parent::label[@data-testid='countries_ar']")))
#countries_cl = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(@class, 'MuiButtonBase-root')]/parent::label[@data-testid='countries_cl']")))
#driver.execute_script("arguments[0].click(); arguments[1].click();", countries_ar countries_cl)
driver.execute_script("arguments[0].click();", countries_ar)
countries_search = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@id='filter-checkboxes-search' and @placeholder='Search countries']")))
countries_search.click()
countries_search.send_keys("Brazil")
time.sleep(1)
countries_br = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(@class, 'MuiButtonBase-root')]/parent::label[@data-testid='countries_br']")))
driver.execute_script("arguments[0].click()", countries_br)

# Wait and see the result
time.sleep(5)

def getPageLinks(list):
    next_page = True

    # Get the page source after it's fully loaded
    html = driver.page_source

    # Pass it to BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    links = [a["href"] for a in soup.find_all('a', {"data-testid": "vintagePageLink"})]

    full_links = ["https://www.vivino.com" + link for link in links]

    list.extend(full_links)

        
    try:
        next_page_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@data-trackingid='buttonDefault'][.//span[.='Next']]")))
        
        try:
            disabled_btn = next_page_btn.get_attribute("aria-disabled")        
            if disabled_btn == "true":
                next_page = False
                return next_page
        except:
            driver.execute_script("arguments[0].click()", next_page_btn)
            time.sleep(5)
            next_page = False
            return next_page
        
    except:
        next_page = False
        return next_page

link_list = []
next_page_available = True

while next_page_available:
    next_page_available = getPageLinks(link_list)

link_df = pd.DataFrame(link_list, columns=["wine_link"])
link_df.to_csv("wine_links.csv", index=False)

driver.quit()

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd

csv = pd.read_csv("rest-links3.csv")
wine_df = pd.DataFrame(csv, columns=["wine_link"]).drop_duplicates()
wine_df["name"] = None
wine_df["year"] = None
wine_df["winery"] = None
wine_df["rating"] = None
wine_df["rating_qty"] = None
wine_df["price"] = None
wine_df["body"] = None
wine_df["tannis"] = None
wine_df["sweetness"] = None
wine_df["acidity"] = None
wine_df["notes"] = None
wine_df["pairings"] = None
wine_df["grapes"] = None
wine_df["region"] = None
wine_df["style"] = None
wine_df["image"] = None

options = webdriver.ChromeOptions()
options.headless = False  
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

first_link = True

# Iterar en las columnas del DF
for index, row in wine_df.iterrows():
    link = row["wine_link"]
    driver.get(link)

    if first_link:
        wait = WebDriverWait(driver, 30)

        # Step 1: click the dropdown menu
        dropdown = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "simpleLabel-module__selectedKey--3ngzL")))
        dropdown.click()
        time.sleep(5)

        # Step 2: select "United States" from the list
        country_option = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@data-value='us']")))
        country_option.click()
        time.sleep(5)

        first_link = False
        
    time.sleep(6)  

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # DATA 1: NOMBRE (name)
    try:
        wine_name_tag = soup.find("a", {"data-cartitemsource": "wine-page-master-link"})
        wine_name = wine_name_tag.get_text(strip=True) if wine_name_tag else None
    except:
        wine_name = None

    # DATA 2: AÑO (year)
    wine_year = None

    try:
        if wine_name_tag:
            parent_div = wine_name_tag.find_parent("div")
            if parent_div:
                year_text = parent_div.get_text(strip=True)
                wine_year = year_text.replace(wine_name, "").strip()
    except:
        wine_year = None

    # DATA 3: BODEGA (WINERY)
    try:
        winery = soup.find("h1").find("div").find("a").find("div").get_text(strip=True)
    except:
        winery = None

    # DATA 4: REVIEWS (Rating)
    try:
        rating_tag = soup.find("a", {"href": "#all_reviews"})
        rating_value = rating_tag.find("div").find("div").get_text(strip=True)
        rating_qty = rating_tag.find_all("div")[-1].get_text(strip=True).split(" ")[0]
    except:
        rating_tag = None
        rating_value = None
        rating_qty = None

    # DATA 5: PRICE (Precio)
    try:
        price = soup.find("span", {"class": "purchaseAvailability__currentPrice--3mO4u"}).get_text(strip=True).replace("$", "")
    except:
        price = None

    # DATA 6: TASTE (Sabor)

    try:
        wine_taste = soup.find_all("tr", {"class": "tasteStructure__tasteCharacteristic--jLtsE"})

        body = None
        tannis = None
        sweetness = None
        acidity = None

        for i, e in enumerate(wine_taste):
            # name = buscar sabor
            name = wine_taste[i].find("td").get_text(strip=True)
            bar = wine_taste[i].find("span", {"class", "indicatorBar__progress--3aXLX"})["style"]
            val = bar.split(";")[1].split(":")[1]
            
            # 6.1 : Light/Bold - Body (Cuerpo)   
            if name == "Light":
                body = val
            
            # 6.2 : Smooth/Tannic - Tannis (Tanicidad)
            elif name == "Smooth":
                tannis = val
            
            # 6.3 : Dry/Sweet - Sweetness (Dulzura)
            elif name == "Dry":
                sweetness = val
            
            # 6.4 : Soft/Acid - Acidity (Acidez)
            elif name == "Soft":
                acidity = val
    except:
        body = None
        tannis = None
        sweetness = None
        acidity = None


    # DATA 7 : NOTE MENTIONS (Menciones de notas)

    try:
        mentions = soup.find_all("div", {"data-testid": "mentions"})

        mentions_dict = {}

        for mention in mentions:
            note = mention.find("span").get_text(strip=True)
            n_mentions = mention.get_text().split(" ")[0]
            mentions_dict[str(note)] = int(n_mentions)
    except:
        mentions_dict = None
    
    # DATA 8 : PAIRINGS (Maridajes)
    try:
        pairing_tags = soup.find("div", {"class": "foodPairing__foodContainer--1bvxM"}).find_all("a", recursive=False)
        
        pairing_list = []

        for i in pairing_tags:
            pairing = i.find("div", {"role": "img"})["aria-label"].lower()
            pairing_list.append(pairing)
    except:
        pairing_tags = None


    # DATA 9 : GRAPES (Uvas)
    try:
        wine_facts = soup.find_all("tr", {"data-testid": "wineFactRow"})
    except:
        wine_facts = None

    try:
        grapes = list(wine_facts[1].find("td").get_text(strip=True).split(","))
    except:
        grapes = None

    try:
    # DATA 10 : REGION
        region = list(wine_facts[2].find("td").get_text(strip=True).split("/"))
    except:
        region = None

    try:
        # DATA 11 : STYLE (Estilo)
        style = wine_facts[3].find("td").get_text()
    except:
        style = None

    try:
        # DATA 12 : WINE IMAGE (Imagen del Vino)
        img = "https:" + soup.find("img", {"class": "wineLabel-module__image--3HOnd"})["src"]
    except:
        img = None

    
    # ÚLTIMO PASO: GUARDAR EN DATAFRAME
    wine_df.at[index, "name"] = wine_name
    wine_df.at[index, "year"] = wine_year
    wine_df.at[index, "winery"] = winery
    wine_df.at[index, "rating"] = rating_value
    wine_df.at[index, "rating_qty"] = rating_qty
    wine_df.at[index, "price"] = price
    wine_df.at[index, "body"] = body
    wine_df.at[index, "tannis"] = tannis
    wine_df.at[index, "sweetness"] = sweetness
    wine_df.at[index, "acidity"] = acidity
    wine_df.at[index, "notes"] = mentions_dict
    wine_df.at[index, "pairings"] = pairing_list
    wine_df.at[index, "grapes"] = grapes
    wine_df.at[index, "region"] = region
    wine_df.at[index, "style"] = style
    wine_df.at[index, "image"] = img

# Guardar el CSV actualizado
wine_df.to_csv("wines1056-2026.csv", index=False)

driver.quit()

wine_df

,wine_link,name,year,winery,rating,rating_qty,price,body,tannis,sweetness,acidity,notes,pairings,grapes,region,style,image
0,https://www.vivino.com/cl-geisse-rose-brut/w/1...,None,None,Cave Amadeu,3.8,5619,26.24,39.776%,None,None,54.945%,"{'red fruit': 56, 'tree fruit': 19, 'citrus': ...","[pork, shellfish, rich fish (salmon, tuna etc)]",[Pinot Noir],"[Brazil, Rio Grande do Sul, Serra Gaúcha]",Brazilian Sparkling,https://images.vivino.com/thumbs/2eG1E4YvQD2lD...
1,https://www.vivino.com/zuccardi-malamado-malbe...,None,None,Zuccardi,3.8,5412,25.95,None,None,None,None,"{'oaky': 146, 'black fruit': 110, 'dried fruit...","[pork, shellfish, rich fish (salmon, tuna etc)]",[Malbec],"[Argentina, Mendoza]",13%,https://images.vivino.com/thumbs/jF7MMUTqSfGFM...
2,https://www.vivino.com/bodega-finca-las-moras-...,Intis Cabernet Sauvignon,2022,Bodega Finca Las Moras,3.5,128,6.75,48.4929675%,58.36217500000001%,18.3055575%,44.01161%,"{'black fruit': 6, 'red fruit': 5, 'spices': 4...","[beef, lamb, poultry]",[Cabernet Sauvignon],"[Argentina, San Juan]",Argentinian Cabernet Sauvignon,https://images.vivino.com/thumbs/iHg5NQpHQYyhv...
3,https://www.vivino.com/ninety-cellars-lot-23-o...,Lot 23 Old Vine Malbec,2022,90+ Cellars,3.5,101,10.99,67.318025%,24.31005%,16.529974999999997%,36.3921%,"{'black fruit': 205, 'oaky': 133, 'red fruit':...","[beef, lamb, pork]",[Malbec],"[Argentina, Mendoza]",Argentinian Mendoza Malbec Red,https://images.vivino.com/thumbs/WSbBwS-_SnWJS...
4,https://www.vivino.com/casarena-estate-caberne...,Estate Cabernet Sauvignon,2019,Casarena,3.5,88,34.99,61.652924999999996%,54.06062%,11.543575%,42.7088075%,"{'black fruit': 41, 'oaky': 22, 'earthy': 8, '...","[beef, lamb, poultry]",[100%Cabernet Sauvignon],"[Argentina, Mendoza, Lujan de Cuyo]",Argentinian Cabernet Sauvignon,https://images.vivino.com/thumbs/4JkmCMpbSc-Jw...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,https://www.vivino.com/etchart-cafayate-terroi...,Cafayate Terroir de Altura Torrontés,2021,Bodegas Etchart,3.9,110,None,47.892224999999996%,None,19.515900000000002%,56.4788025%,"{'tree fruit': 3, 'red fruit': 1, 'floral': 1}","[vegetarian, spicy food, aperitif]",[Torrontés],"[Argentina, Salta, Calchaqui Valley, Cafayate ...",Argentinian Torrontés,https://images.vivino.com/thumbs/zbf30yF2Q_6j_...
966,https://www.vivino.com/siesta-cabernet-sauvign...,Cabernet Sauvignon,2011,Siesta,3.9,108,None,65.158975%,54.17996%,13.757972500000001%,41.570055%,"{'oaky': 22, 'black fruit': 14, 'red fruit': 1...","[beef, lamb, poultry]",[100%Cabernet Sauvignon],"[Argentina, Mendoza]",Argentinian Cabernet Sauvignon,https://images.vivino.com/thumbs/hz2-G9-7TyGMC...
967,https://www.vivino.com/gen-del-alma-jijiji-mal...,JIJIJI Malbec - Pinot Noir,2022,Gen del Alma,3.9,105,22.66,63.11037999999999%,45.074575%,21.187649999999998%,58.78908%,"{'red fruit': 60, 'black fruit': 27, 'earthy':...","[beef, lamb, pork, poultry, mushrooms, blue ch...","[Malbec, Pinot Noir]","[Argentina, Mendoza, Uco Valley, Tupungato]",Argentinian Malbec Red Blend,https://images.vivino.com/thumbs/hqHY7FTaRoucR...
968,https://www.vivino.com/terrazas-de-los-andes-h...,High Altitude Vineyards Malbec,2021,Terrazas de los Andes,3.9,103,None,62.6411375%,31.788999999999994%,18.47617%,37.5830825%,"{'oaky': 1756, 'black fruit': 1668, 'earthy': ...","[beef, lamb]",[100%Malbec],"[Argentina, Mendoza, Lujan de Cuyo]",Argentinian Lujan de Cuyo Malbec Red,https://images.vivino.com/thumbs/xaqIkl7ERfa2e...


In [32]:
wine_df.loc[wine_df["name"].notna()]

,wine_link,name,year,winery,rating,rating_qty,price,body,tannis,sweetness,acidity,notes,pairings,grapes,region,style,image
2,https://www.vivino.com/altos-las-hormigas-blan...,Blanco,2024,Altos Las Hormigas,3.7,based,16.88,44.5260575%,None,9.288775000000001%,46.5580025%,"{'citrus': 11, 'tree fruit': 10, 'earthy': 7, ...","[pork, shellfish, vegetarian, mature and hard ...","[Chenin Blanc, Sémillon, Pedro Giménez]","[Argentina, Mendoza]",Argentinian White,https://images.vivino.com/thumbs/WtUTz_LiSOyAp...
3,https://www.vivino.com/altos-del-plata-malbec/...,Malbec,2022,Altos del Plata,3.7,484,11.54,61.005632500000004%,28.626800000000003%,18.630557499999995%,37.4329075%,"{'oaky': 639, 'black fruit': 578, 'earthy': 34...","[beef, lamb]",[Malbec],"[Argentina, Mendoza, Lujan de Cuyo]",Argentinian Lujan de Cuyo Malbec Red,https://images.vivino.com/thumbs/5FixNfhtRui_n...
4,https://www.vivino.com/matias-riccitelli-the-a...,The Apple Doesn't Fall Far From The Tree Torro...,2024,Matías Riccitelli,3.7,based,20.26,47.043287500000005%,None,11.772762500000002%,56.6847075%,"{'tree fruit': 37, 'citrus': 30, 'earthy': 17,...","[vegetarian, spicy food, aperitif]",[Torrontés],"[Argentina, Mendoza]",Argentinian Torrontés,https://images.vivino.com/thumbs/z2ZdkSD3RHaTr...
5,https://www.vivino.com/ar-pascual-toso-malbec/...,Malbec,2021,Pascual Toso,3.7,469,12.99,60.639195%,24.281475000000004%,17.0916225%,35.30237%,"{'oaky': 532, 'black fruit': 425, 'earthy': 23...","[beef, lamb, pork]",[100%Malbec],"[Argentina, Mendoza]",Argentinian Mendoza Malbec Red,https://images.vivino.com/thumbs/toIDvGr7Rqyxl...
6,https://www.vivino.com/don-rodolfo-tannat/w/18...,Tannat,2021,Don Rodolfo,3.7,based,16.79,None,None,None,None,"{'black fruit': 49, 'oaky': 40, 'earthy': 31, ...","[beef, lamb, pork]",[Tannat],"[Argentina, Mendoza]",Contains sulfites,https://images.vivino.com/thumbs/_cLDvOCOQoOci...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,https://www.vivino.com/bressia-sylvestra-torro...,Sylvestra Torrontés,2022,Bressia,3.5,based,22.04,45.943949999999994%,None,3.9985225000000018%,55.0206525%,"{'tree fruit': 11, 'citrus': 5, 'tropical': 4,...","[vegetarian, spicy food, aperitif]",[Torrontés],"[Argentina, Mendoza]",Argentinian Torrontés,https://images.vivino.com/thumbs/tfBkFm_fRrecU...
187,https://www.vivino.com/vina-cobos-felino-merlo...,Felino Merlot,2011,Viña Cobos,3.5,257,25,63.5295625%,27.690875%,24.391887500000003%,15.339949999999998%,"{'oaky': 19, 'black fruit': 13, 'red fruit': 1...","[beef, lamb, veal]",[Merlot],"[Argentina, Mendoza]",Argentinian Merlot,https://images.vivino.com/thumbs/PAOSfhU0RaGk0...
188,https://www.vivino.com/bodega-finca-las-moras-...,Intis Malbec,2022,Bodega Finca Las Moras,3.5,248,6.75,49.051384999999996%,30.540594999999996%,23.930882500000003%,39.75299%,"{'black fruit': 22, 'red fruit': 16, 'oaky': 7...","[beef, lamb, poultry]",[Malbec],"[Argentina, San Juan]",Argentinian Malbec,https://images.vivino.com/thumbs/OQcrOeL5R7K1d...
189,https://www.vivino.com/sur-de-los-andes-gran-r...,Gran Reserva Malbec,2020,Sur de Los Andes,3.5,based,22.04,68.94090750000001%,19.078785000000003%,11.112119999999997%,30.489395000000002%,"{'black fruit': 2, 'spices': 1, 'oaky': 1}","[beef, lamb, pork]",[Malbec],"[Argentina, Mendoza]",Argentinian Mendoza Malbec Red,https://images.vivino.com/thumbs/02oew1s7pno9n...


In [33]:
wine = wine_df.loc[wine_df["name"].notna()]
wine.to_csv("wines867-1055.csv", index=False)

In [34]:
rest = wine_df.loc[wine_df["name"].isna()]["wine_link"]
rest.to_csv("rest-links3.csv", index=False)